# Pipelines with different encodings for categorical data
(with Random Forest)

In [1]:
import pandas as pd

In [2]:
random_state = 42

# Load data

In [3]:
train_values = pd.read_csv('../data/train_values.csv', index_col='building_id')
train_values.head()

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
building_id,,,,,,,,,,,,,,,,,,,,,
802906,6,487,12198,2,30,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
28830,8,900,2812,2,10,8,7,o,r,n,...,0,0,0,0,0,0,0,0,0,0
94947,21,363,8973,2,10,5,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
590882,22,418,10694,2,10,6,5,t,r,n,...,0,0,0,0,0,0,0,0,0,0
201944,11,131,1488,3,30,8,9,t,r,n,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_labels = pd.read_csv('../data/train_labels.csv', index_col='building_id')

,damage_grade
building_id,
802906,3
28830,2
94947,3
590882,2
201944,3


# Explore features

In [5]:
# Categorical columns = 
categorical_columns = [c for c in train_values.select_dtypes(include=['object'])]
for c in categorical_columns:
    print(c)

land_surface_condition
foundation_type
roof_type
ground_floor_type
other_floor_type
position
plan_configuration
legal_ownership_status


In [6]:
numerical_columns = list(set(train_values.columns) - set(categorical_columns))
numerical_columns

['has_superstructure_stone_flag',
 'has_superstructure_rc_engineered',
 'count_floors_pre_eq',
 'has_superstructure_mud_mortar_stone',
 'has_superstructure_cement_mortar_stone',
 'has_secondary_use_gov_office',
 'has_secondary_use_rental',
 'has_secondary_use_hotel',
 'height_percentage',
 'has_secondary_use_school',
 'has_superstructure_timber',
 'has_secondary_use_industry',
 'has_superstructure_other',
 'has_secondary_use',
 'age',
 'has_secondary_use_other',
 'area_percentage',
 'has_superstructure_rc_non_engineered',
 'has_secondary_use_agriculture',
 'geo_level_2_id',
 'has_superstructure_adobe_mud',
 'geo_level_1_id',
 'has_secondary_use_institution',
 'has_secondary_use_use_police',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_bamboo',
 'has_secondary_use_health_post',
 'count_families',
 'geo_level_3_id',
 'has_superstructure_cement_mortar_brick']

### Drop duplicates

In [8]:
duplicate_index_mask = train_values.index.duplicated(keep='first')
train_values = train_values[~duplicate_index_mask]
train_labels = train_labels[~duplicate_index_mask]

### Drop the index column

In [9]:
# Drop building_id (index) from X and y
train_values.reset_index(drop=True, inplace=True)
train_labels.reset_index(drop=True, inplace=True)
# Convert y to a Series instead of a DataFrame
train_labels = train_labels['damage_grade']


In [10]:
# convert geo data to categorical
def fixup_geo_data(data):
    geo_cols = ['geo_level_1_id', 'geo_level_2_id', 'geo_level_3_id']
    for c in geo_cols:
        data[c] = data[c].astype('object')
fixup_geo_data(train_values)

## Split the data

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_values, train_labels, train_size=0.8, test_size=0.2, random_state=random_state)

All possible values from all categorical columns are present in the train set

## Encode categorical values

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, TargetEncoder, OneHotEncoder

In [13]:
def build_preprocessor_drop_cat():
    return ColumnTransformer(
    transformers=[
        ('cat', 'drop', categorical_columns),
        ('numerical', 'passthrough', numerical_columns),
    ])

def build_preprocessor_ordinal_enc():
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
    ])

    # Bundle preprocessing pipeline
    return ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_columns),
            ("numerical", "passthrough", numerical_columns),
        ])

def build_preprocessor_target_enc():
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('target', TargetEncoder(target_type='continuous'))
    ])

    # Bundle preprocessing pipeline
    return ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_columns),
            ("numerical", "passthrough", numerical_columns),
        ])

def build_preprocessor_one_hot_enc():
    # Preprocessing for categorical data
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder())
    ])

    # Bundle preprocessing pipeline
    return ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_columns),
            ("numerical", "passthrough", numerical_columns),
        ])

In [14]:
preprocessors = dict(
    drop_cat=build_preprocessor_drop_cat(),
    ordinal_enc_cat=build_preprocessor_ordinal_enc(),
    target_enc_cat=build_preprocessor_target_enc(),
    one_hot_encoder=build_preprocessor_one_hot_enc())

In [15]:
# the model
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators=100, random_state=random_state)

In [16]:
from dataclasses import dataclass

@dataclass 
class EvaluatedModel:
    name: str
    pipeline: Pipeline
    score: float

In [17]:
def fit_model_via_pipeline(name, pipeline, model):
    clf = Pipeline(
       steps=[('preprocessor', pipeline),
              ('model', model)])

    X = X_train.copy()
    y = y_train.copy()
    clf.fit(X, y)
    score = clf.score(X_valid.copy(), y_valid.copy())
    print(f"Evaluated model: {name}, score: {score}")
    return EvaluatedModel(name=name, pipeline=clf, score=score)

In [18]:
fitted_pipelines = []
for key in preprocessors:
    model = fit_model_via_pipeline(key, preprocessors[key], random_forest)
    fitted_pipelines.append(model)

Evaluated model: drop_cat, score: 0.7132633679323114
Evaluated model: ordinal_enc_cat, score: 0.7204006062815372
Evaluated model: target_enc_cat, score: 0.7272692388864374
Evaluated model: one_hot_encoder, score: 0.712534295197713


In [19]:
best_pipeline = max(fitted_pipelines, key=lambda x:x.score)
#best_preprocessor_key = 'target_enc_cat'
print(f"The best preprocessor: {best_pipeline.name}, score: {best_pipeline.score}")

The best preprocessor: target_enc_cat, score: 0.7272692388864374
